<a href="https://colab.research.google.com/github/ananthakrishnagopal/Computational-Geometry/blob/main/Line_Sweep_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from dataclasses import dataclass
from enum import Enum
from sortedcontainers import SortedDict
import math


In [4]:

# === Data Structures ===
@dataclass
class Point:
    """2D point with x, y coordinates."""
    x: float
    y: float

    def __lt__(self, other):
        """Sort by y (descending), then x (ascending)."""
        if not math.isclose(self.y, other.y):
            return self.y > other.y
        return self.x < other.x

    def __eq__(self, other):
        """Equal if x, y are close within 1e-10."""
        if not isinstance(other, Point):
            return NotImplemented
        return math.isclose(self.x, other.x, abs_tol=1e-10) and math.isclose(self.y, other.y, abs_tol=1e-10)

    def __hash__(self):
        """Hash rounded coordinates for precision."""
        return hash((round(self.x, 10), round(self.y, 10)))

    def __repr__(self):
        return f"Point({self.x:.3f}, {self.y:.3f})"

@dataclass
class Segment:
    """Line segment with upper (u) and lower (v) endpoints, and an ID."""
    u: Point
    v: Point
    id: int = 0

    def __post_init__(self):
        """Ensure u is the upper endpoint (higher y, or same y with smaller x)."""
        if self.v < self.u:
            self.u, self.v = self.v, self.u

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(other, Segment) and self.id == other.id

    def __repr__(self):
        return f"Seg{self.id}({self.u} -> {self.v})"

    def x_at_y(self, y):
        """Get x-coordinate where segment intersects horizontal line at y."""
        if math.isclose(self.u.y, self.v.y):  # Horizontal
            return self.u.x
        if math.isclose(self.u.x, self.v.x):  # Vertical
            return self.u.x
        t = (y - self.u.y) / (self.v.y - self.u.y)  # Linear interpolation
        return self.u.x + t * (self.v.x - self.u.x)

    def contains_point_interior(self, p):
        """Check if point p is strictly inside segment (not at endpoints)."""
        if p == self.u or p == self.v:
            return False
        cross = (p.y - self.u.y) * (self.v.x - self.u.x) - (p.x - self.u.x) * (self.v.y - self.u.y)
        if not math.isclose(cross, 0, abs_tol=1e-10):
            return False
        min_x, max_x = min(self.u.x, self.v.x), max(self.u.x, self.v.x)
        min_y, max_y = min(self.u.y, self.v.y), max(self.u.y, self.v.y)
        return (min_x < p.x < max_x or math.isclose(min_x, max_x)) and \
               (min_y < p.y < max_y or math.isclose(min_y, max_y))

    def intersects(self, other):
        """Find intersection point with another segment, if it exists."""
        if self == other or {self.u, self.v} & {other.u, other.v}:  # Same or shared endpoint
            return None
        x1, y1, x2, y2 = self.u.x, self.u.y, self.v.x, self.v.y
        x3, y3, x4, y4 = other.u.x, other.u.y, other.v.x, other.v.y
        denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
        if math.isclose(denom, 0, abs_tol=1e-10):  # Parallel
            return None
        t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denom
        u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denom
        if 0 < t < 1 and 0 < u < 1:  # Intersection inside both segments
            return Point(x1 + t * (x2 - x1), y1 + t * (y2 - y1))
        return None

class EventType(Enum):
    START = 1
    END = 2
    INTERSECTION = 3

@dataclass
class Event:
    point: Point
    segments: set
    event_type: EventType

    def __lt__(self, other):
        return self.point < other.point

    def __repr__(self):
        return f"{self.event_type.name}@{self.point} segs={[s.id for s in self.segments]}"

In [6]:
# === Main Algorithm ===
def find_intersections(segments):
    """Find all intersection points among line segments using Bentley-Ottmann."""
    # Step 1: Initialize segments with IDs and normalize endpoints
    for i, seg in enumerate(segments):
        seg.id = i
        seg.__post_init__()

    # Step 2: Create event queue with start and end points
    event_queue = SortedDict()
    for seg in segments:
        for p, etype in [(seg.u, EventType.START), (seg.v, EventType.END)]:
            if p not in event_queue:
                event_queue[p] = Event(p, set(), etype)
            event_queue[p].segments.add(seg)

    # Step 3: Initialize status (segments intersecting sweep line) and intersections
    status = SortedDict()  # Ordered by x at current y
    intersections = set()
    current_y = float('inf')

    def get_status_key(seg, y):
        """Sort key for status: x-coordinate at y, then segment ID."""
        return (seg.x_at_y(y), seg.id)

    def add_intersection_event(seg1, seg2, sweep_y):
        """Add intersection event if segments intersect below sweep line."""
        if p := seg1.intersects(seg2):
            if p.y < sweep_y or (math.isclose(p.y, sweep_y) and p.x > current_x):
                if p not in event_queue:
                    event_queue[p] = Event(p, {seg1, seg2}, EventType.INTERSECTION)
                else:
                    event_queue[p].segments.update({seg1, seg2})

    # Step 4: Process events
    while event_queue:
        point, event = event_queue.popitem(0)
        current_y, current_x = point.y, point.x

        # Classify segments
        starting = {s for s in event.segments if s.u == point}
        ending = {s for s in event.segments if s.v == point}
        interior = {s for s in event.segments if s.contains_point_interior(point)}

        # If multiple segments meet, it's an intersection
        all_segs = starting | ending | interior
        if len(all_segs) > 1:
            intersections.add(point)

        # Update status: remove ending/interior, add starting/interior
        for seg in ending | interior:
            status.pop(get_status_key(seg, current_y), None)
        for seg in starting | interior:
            status[get_status_key(seg, current_y - 1e-10)] = seg

        # Check neighbors for new intersections
        if starting or interior:
            current_segs = list(status.values())
            if len(current_segs) >= 2:
                current_segs.sort(key=lambda s: s.x_at_y(current_y - 1e-10))
                added = [i for i, s in enumerate(current_segs) if s in starting | interior]
                if added:
                    left_idx = min(added)
                    right_idx = max(added)
                    if left_idx > 0:
                        add_intersection_event(current_segs[left_idx - 1], current_segs[left_idx], current_y)
                    if right_idx < len(current_segs) - 1:
                        add_intersection_event(current_segs[right_idx], current_segs[right_idx + 1], current_y)
        else:
            current_segs = list(status.values())
            if len(current_segs) >= 2:
                current_segs.sort(key=lambda s: s.x_at_y(current_y - 1e-10))
                for i in range(len(current_segs) - 1):
                    add_intersection_event(current_segs[i], current_segs[i + 1], current_y)

    return intersections


In [7]:

# === Test Cases ===
if __name__ == "__main__":
    # Test 1: Two segments forming an X
    segments1 = [
        Segment(Point(0, 2), Point(2, 0), 0),
        Segment(Point(0, 0), Point(2, 2), 1)
    ]
    print("Test 1:", find_intersections(segments1))  # Expected: {Point(1.0, 1.0)}

    # Test 2: Parallel horizontal segments
    segments2 = [
        Segment(Point(0, 3), Point(1, 3), 0),
        Segment(Point(0, 1), Point(1, 1), 1)
    ]
    print("Test 2:", find_intersections(segments2))  # Expected: set()

    # Test 3: Three segments meeting at one point
    segments3 = [
        Segment(Point(0, 0), Point(2, 2), 0),
        Segment(Point(2, 0), Point(0, 2), 1),
        Segment(Point(1, 0), Point(1, 2), 2)
    ]
    print("Test 3:", find_intersections(segments3))  # Expected: {Point(1.0, 1.0)}
# === Test Cases ===
if __name__ == "__main__":
    # Test 1: Two segments forming an X
    segments1 = [
        Segment(Point(0, 2), Point(2, 0), 0),
        Segment(Point(0, 0), Point(2, 2), 1)
    ]
    print("Test 1:", find_intersections(segments1))  # Expected: {Point(1.0, 1.0)}

    # Test 2: Parallel horizontal segments
    segments2 = [
        Segment(Point(0, 3), Point(1, 3), 0),
        Segment(Point(0, 1), Point(1, 1), 1)
    ]
    print("Test 2:", find_intersections(segments2))  # Expected: set()

    # Test 3: Three segments meeting at one point
    segments3 = [
        Segment(Point(0, 0), Point(2, 2), 0),
        Segment(Point(2, 0), Point(0, 2), 1),
        Segment(Point(1, 0), Point(1, 2), 2)
    ]
    print("Test 3:", find_intersections(segments3))  # Expected: {Point(1.0, 1.0)}

Test 1: {Point(1.000, 1.000)}
Test 2: set()
Test 3: {Point(1.000, 1.000)}
Test 1: {Point(1.000, 1.000)}
Test 2: set()
Test 3: {Point(1.000, 1.000)}
